In [0]:
import sys
print(sys.version)

CURR_DIR = '/content/drive/My Drive/google_colab_work/detailed_probabilistic_robotics/'
sys.path.append(CURR_DIR)
sys.path.append(CURR_DIR + 'scripts/')

3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]


In [0]:
from robot import *

3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
class PsiCamera(Camera):
    def data(self, cam_pose, orientation_std=math.radians(2.0)):
        observed = []
        for landmark in self.map.landmarks:
            z = self.observation_function(cam_pose, landmark.pos)
            z = self.phantom(cam_pose, z)
            z = self.occlusion(z)
            z = self.oversight(z)

            dx = cam_pose[0] - landmark.pos[0]
            dy = cam_pose[1] - landmark.pos[1]
            psi = scipy.stats.norm.rvs(loc=math.atan2(dy, dx), scale=orientation_std)

            if self.visible(z):
                z += self.bias(z) + self.noise(z)
                obs = ([z[0], z[1], psi], landmark.id)
                observed.append(obs)
                
        self.lastdata = observed
        return observed

In [0]:
class LoggerAgent(Agent):
    def __init__(self, init_pose, nu, omega, time_interval):
        super().__init__(nu, omega)
        self.pose = init_pose
        self.time_interval = time_interval
        self.log = open('graph_slam_log.txt', 'w')
        self.log.write('time_interval: {}\n'.format(time_interval))
        self.step = 0

    def decision(self, obs):
        self.log.write('u {} {} {}\n'.format(self.step, self.nu, self.omega)) ### [s], [m/s], [rad/s]
        self.log.write('x {} {} {} {}\n'.format(self.step, *self.pose)) ### [s], [m], [m], [rad]
        for z, obs_id in obs:
            self.log.write('z {} {} {} {} {}\n'.format(self.step, obs_id, *z)) ### [s], ID, [m], [rad], [rad]
            
        self.step += 1
        self.log.flush()

        self.pose = IdealRobot.state_transition(self.pose, self.nu, self.omega, self.time_interval)
        return self.nu, self.omega

In [0]:
time_span = 180
time_interval = 3
world = World(time_span, time_interval, debug=False)

m = Map()
for landmark in [(-4,2), (2,-3), (3,3), (0,4), (1,1), (-3,-1)]:
    m.append_landmark(Landmark(*landmark))
world.append(m)

init_pose = np.array([0, -3, math.radians(0.0)])
nu = 0.2
omega = math.radians(5.0)
circling_agent = LoggerAgent(init_pose, nu, omega, time_interval)

robot = Robot(init_pose, agent=circling_agent, sensor=PsiCamera(m), color='red')
world.append(robot)

world.draw()
world.ani

<IPython.core.display.Javascript object>